In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# CIFAR-10 has 10 classes
num_classes = 10

# Image Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalization
])

# Download and load the CIFAR-10 data
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform)

# Data Loader
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Define a GoogLeNet model for CIFAR-10
class GoogLeNetCIFAR10(models.GoogLeNet):
    def __init__(self, num_classes=10):
        super(GoogLeNetCIFAR10, self).__init__(num_classes=num_classes, aux_logits=False, init_weights=True)
        # Redefine the first convolutional layer to accept CIFAR-10's 32x32 pixel images
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # Follow the forward pass of the GoogLeNet model
        return super(GoogLeNetCIFAR10, self).forward(x)

# Instantiate the model and transfer it to the GPU
model = GoogLeNetCIFAR10(num_classes=num_classes).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to train the model
def train(model, criterion, optimizer, train_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            # Transfer inputs and labels to the GPU
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100}')
                running_loss = 0.0

# Function to test the model
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            # Transfer images and labels to the GPU
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Start training
train(model, criterion, optimizer, train_loader, epochs=10)

# Test the model
accuracy = test(model, test_loader)
print(f'Accuracy of the model on the 10000 test images: {accuracy}%')


Using device: cuda:0


100%|██████████| 170498071/170498071 [00:13<00:00, 12579363.55it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Epoch 1, Batch 100, Loss: 1.908914761543274
Epoch 1, Batch 200, Loss: 1.6881796860694884
Epoch 1, Batch 300, Loss: 1.5614552509784698
Epoch 1, Batch 400, Loss: 1.476861379146576
Epoch 1, Batch 500, Loss: 1.3676432633399964
Epoch 1, Batch 600, Loss: 1.2921091383695602
Epoch 1, Batch 700, Loss: 1.2211811923980713
Epoch 2, Batch 100, Loss: 1.116407887339592
Epoch 2, Batch 200, Loss: 1.046716919541359
Epoch 2, Batch 300, Loss: 1.0427150011062623
Epoch 2, Batch 400, Loss: 0.992351871728897
Epoch 2, Batch 500, Loss: 0.9412319147586823
Epoch 2, Batch 600, Loss: 0.9406729835271835
Epoch 2, Batch 700, Loss: 0.8822331881523132
Epoch 3, Batch 100, Loss: 0.8083757004141807
Epoch 3, Batch 200, Loss: 0.8033378323912621
Epoch 3, Batch 300, Loss: 0.7827082127332687
Epoch 3, Batch 400, Loss: 0.7645208638906479
Epoch 3, Batch 500, Loss: 0.742219472527504
Epoch 3, Batch 600, Loss: 0.7189898347854614
Epoch 3, Batch 700, Loss: 0.7046088826656342
Epoch 4, B